In [1]:
import os
import sys
import dotenv
from pathlib import Path
from IPython.display import Markdown as md

sys.path.append(os.path.expanduser("~/source/appland/SWE-bench"))

project_dir = Path(os.path.expanduser("~")) / "source" / "appland" / "appmap-js"
# Expand home path to full path
os.chdir(project_dir)

from appmap.navie.editor import Editor
from appmap.navie.format_instructions import xml_format_instructions
from appmap.navie.fences import extract_fenced_content

dotenv.load_dotenv(".env.notebook", override=True)
plan_dir = project_dir / "notebooks" / "transform_xml_to_markdown"

In [2]:
class_template = """
class XMLToMarkdownTransform {
  inMarkdown = false;

  transform(chunk: string, callback: (chunk: string) => void) {


  }
}
"""

In [8]:
import re

code_raw = Editor(str(plan_dir / "generate")).ask(
    f"""@generate
    
Code suggestions are emitted, mixed with other Markdown, in string "chunks" one by one.

The expected format of the code suggetions is:

```xml
<code>
    <language>language name</language>
    <file>path/to/file</file>
    <content><![CDATA[
        ( Updated code content )
    ]]></content>
</code>
```

Template code is provided of a transformer.

Update the transformer accept one chunk at a time. The transformer should keep track of 
whether its in a code block or not. When it enters a code block, and has the filename and language, 
it should emit a Markdown / HTML comment with the file name, then begin a fenced code block with the
language. Then as it receives more tokens, emit those tokens until the code block ends. When the code
block ends, emit the closing fence.

When the transformer is not in a code block, it should emit the tokens as is.

DO NOT match strictly against one chunk at a time, because critical output may be split across chunks.
Accumulate the tokens until you have enough to emit a complete output. 

You MAY assume that tags will not be split across lines.

Template code:

{class_template}
""",
)

md(code_raw)



  Using cached answer
  Output is available at /Users/kgilpin/source/appland/appmap-js/notebooks/transform_xml_to_markdown/generate/ask/ask.md


```xml
<code>
    <language>typescript</language>
    <file>packages/navie/src/lib/XMLToMarkdownTransform.ts</file>
    <content><![CDATA[
export default class XMLMarkdownTransform {
  private inMarkdown = false;
  private buffer = '';

  transform(chunk: string, callback: (chunk: string) => void) {
    this.buffer += chunk;

    while (this.buffer) {
      if (!this.inMarkdown) {
        const startMatch = this.buffer.match(/<code>\s*<language>(.*?)<\/language>\s*<file>(.*?)<\/file>\s*<content><!\[CDATA\[/s);
        if (startMatch) {
          callback(`<!-- ${startMatch[2]} -->\n\`\`\`${startMatch[1]}\n`);
          this.buffer = this.buffer.slice(startMatch.index! + startMatch[0].length);
          this.inMarkdown = true;
        } else {
          callback(this.buffer);
          this.buffer = '';
        }
      } else {
        const endMatch = this.buffer.match(/\]\]><\/content>\s*<\/code>/s);
        if (endMatch) {
          callback(this.buffer.slice(0, endMatch.index) + '\n```\n');
          this.buffer = this.buffer.slice(endMatch.index! + endMatch[0].length);
          this.inMarkdown = false;
        } else {
          callback(this.buffer);
          this.buffer = '';
          break;
        }
      }
    }
  }
}
    ]]></content>
</code>
```

In [10]:
files = [ "packages/navie/src/lib/xml-markdown-transformer.ts" ]

test_code_raw = Editor(str(plan_dir / "test")).generate(
    f"""/include=spec
Generate test case code for the transformer.

Focus on testing the transformer's ability to accept chunks of the content one by one,
interpret them correctly, and emit the correct output.

Test cases should be written to send a defined sequence of chunks to the transformer, and
then check the output.


""",
    files=files,
  
)

md(test_code_raw)


  Output is available at /Users/kgilpin/source/appland/appmap-js/notebooks/transform_xml_to_markdown/test/generate/generate.md


To properly test the `XMLMarkdownTransformer`, we need to verify that it can process chunks of input and produce the expected output. We'll write some Jest test cases for this transformer. These test cases will cover various scenarios, such as:

1. Properly identifying and transforming the start and end of a <code> section.
2. Handling buffers that contain both complete and incomplete <code> sections.
3. Ensuring the transformer processes multiple chunks correctly.

Let's create a Jest test file for this:

```javascript
// File: packages/navie/src/lib/xml-markdown-transformer.spec.ts
import XMLMarkdownTransformer from './xml-markdown-transformer';

describe('XMLMarkdownTransformer', () => {
  let transformer: XMLMarkdownTransformer;
  let callback: jest.Mock;

  beforeEach(() => {
    transformer = new XMLMarkdownTransformer();
    callback = jest.fn();
  });

  test('should transform a complete code block in one chunk', () => {
    const input = '<code> <language>javascript</language> <file>example.js</file> <content><![CDATA[console.log("Hello, World!");]]></content> </code>';
    transformer.transform(input, callback);

    expect(callback.mock.calls).toEqual([
      ['<!-- example.js -->\n```javascript\n'],
      ['console.log("Hello, World!");\n```\n']
    ]);
  });

  test('should transform code block across multiple chunks', () => {
    const chunk1 = '<code> <language>javascript</language> <file>example.js</file> <content><![CDATA[console.log("';
    const chunk2 = 'Hello, World!");]]></content> </code>';

    transformer.transform(chunk1, callback);
    transformer.transform(chunk2, callback);

    expect(callback.mock.calls).toEqual([
      ['<!-- example.js -->\n```javascript\n'],
      ['console.log("Hello, World!");\n```\n']
    ]);
  });

  test('should handle incomplete code blocks gracefully', () => {
    const input = '<code> <language>javascript</language> <file>example.js</file> <content><![CDATA[console.log("Hello, World!");';
    transformer.transform(input, callback);

    expect(callback).toHaveBeenCalledWith(input);
  });

  test('should transform multiple code blocks in sequence', () => {
    const chunk1 = '<code> <language>javascript</language> <file>example.js</file> <content><![CDATA[console.log("Hello, World!");]]></content> </code>';
    const chunk2 = '<code> <language>python</language> <file>example2.py</file> <content><![CDATA[print("Hello, World!")]]></content> </code>';
    
    transformer.transform(chunk1, callback);
    transformer.transform(chunk2, callback);

    expect(callback.mock.calls).toEqual([
      ['<!-- example.js -->\n```javascript\n'],
      ['console.log("Hello, World!");\n```\n'],
      ['<!-- example2.py -->\n```python\n'],
      ['print("Hello, World!")\n```\n']
    ]);
  });

  test('should handle non-code content correctly', () => {
    const input = 'This is some regular text';
    transformer.transform(input, callback);

    expect(callback).toHaveBeenCalledWith(input);
  });
});
```

Add this `xml-markdown-transformer.spec.ts` file to `packages/navie/src/lib/` directory:

```plaintext
packages/navie/src/lib/xml-markdown-transformer.spec.ts
```

These test cases should cover a variety of scenarios, ensuring that the `XMLMarkdownTransformer` correctly processes chunks of input data and emits the expected output. Run these tests with Jest to verify the functionality.

Ensure that your Jest configuration supports TypeScript. You should already have necessary configurations based on the provided snippet:

```javascript
// File: packages/cli/jest.config.js
/** @type {import('ts-jest/dist/types').InitialOptionsTsJest} */
module.exports = {
  preset: 'ts-jest',
  testEnvironment: 'node',
  testTimeout: parseInt(process.env.TEST_TIMEOUT, 10) || 5000,
  silent: process.env.TEST_SILENT !== 'false',
  restoreMocks: true,
  maxWorkers: 1,
};

// And for other packages e.g., components
module.exports = {
  setupFilesAfterEnv: ['jest-extended'],
  preset: '@vue/cli-plugin-unit-jest/presets/typescript-and-babel',
  transformIgnorePatterns: ['../../node_modules/(?!dom-to-svg)'],
  transform: {
    '^.+\\.svg$': '<rootDir>/tests/unit/support/svgTransform.js',
  },
  moduleNameMapper: {
    'd3-flame-graph': 'd3-flame-graph',
    '^d3-(.*)$': 'd3-$1/dist/d3-$1',
  },
};

```

With this configuration, you should be able to run TypeScript tests using Jest.